In [4]:
pacman::p_load(BiocManager,Biostrings,Rsubread)

In [5]:
R.version

               _                           
platform       x86_64-conda-linux-gnu      
arch           x86_64                      
os             linux-gnu                   
system         x86_64, linux-gnu           
status                                     
major          4                           
minor          1.0                         
year           2021                        
month          05                          
day            18                          
svn rev        80317                       
language       R                           
version.string R version 4.1.0 (2021-05-18)
nickname       Camp Pontanezen             

In [6]:
assemblies_dir = "../../mmetsp_metatranscriptomes/mock_assemblies"
rawreads_dir = "../../mmetsp_metatranscriptomes/mock_raw_reads"

instructions_file = read.csv("../../mmetsp_metatranscriptomes/communities.csv")

In [5]:
fasta_output = read.fasta(file = file.path(assemblies_dir[1],list.files(assemblies_dir[1])))

ERROR: Error in read.fasta(file = file.path(assemblies_dir[1], list.files(assemblies_dir[1]))): could not find function "read.fasta"


In [22]:
lengths_assemblies = c(85125,128713,161206,105210,183562)
counter = 1
for (assembly_dir in list.files(assemblies_dir)) {
    fasta_file = file.path(assemblies_dir,assembly_dir,list.files(file.path(assemblies_dir,assembly_dir)))
    assembly_stub = strsplit(assembly_dir,"\\.")[1]
    output_name = file.path(rawreads_dir,assembly_stub)
    dir.create(output_name, showWarnings = FALSE, recursive=TRUE)
    scanned_info = Rsubread::scanFasta(

            # the file containing the transcript database
            fasta_file,

            # manipulating transcript names
            simplify.transcript.names = FALSE,

            # miscellaneous options
            quiet = FALSE)
    
    nsequences <- nrow(scanned_info) - sum(scanned_info$Duplicate)

    # Assign a random TPM value to each non-duplicated transcript sequence
    expressionlevels <- rep(0, nrow(scanned_info))
    expressionlevels[!scanned_info$Duplicate] <- rexp(nsequences) # exponential distribution
    transcript_info = Rsubread::simReads(

        # the transcript database and the wanted abundances
        fasta_file,
        #expression.levels,
        expressionlevels,

        # the name of the output
        file.path(output_name,paste0(assembly_stub)),

        # options on the output
        library.size = 1000000,
        read.length = 75,
        # logical indicating if the true mapping location of reads or
        # read-pairs should be encoded into the read names
        truth.in.read.names = TRUE,

        # simulating sequencing errors
        simulate.sequencing.error = TRUE,
        quality.reference = NULL,

        # parameters for generating paired-end reads.
        paired.end = TRUE,
        fragment.length.min = 100,
        fragment.length.max = 500,
        fragment.length.mean = 180,
        fragment.length.sd = 40,

        # manipulating transcript names
        simplify.transcript.names = FALSE)
    counter = counter + 1
}